# Load data

In [ ]:
import os
import pandas as pd
import sqlite3

import matplotlib.pyplot as plt

def has_swear_word(text):
    if text == None:
        return False
    words = text.split()
    for w in words:
        for sw in swear_words:
            if sw.lower() == w.lower():
                return True
    return False

#Get swear word list
with open("../src/swear_word_list.txt", 'r') as f:
    swear_words = [line.strip() for line in f]

con = sqlite3.connect("../data/database.db")

track_df = pd.read_sql("SELECT * FROM Track", con=con)
artist_df = pd.read_sql("SELECT * from Artist", con=con)
album_df = pd.read_sql("SELECT * FROM Album", con=con)

# Drop null or empty values

In [ ]:
track_df.replace("", float("NaN"), inplace=True)
track_df.dropna(inplace=True)

# Handle escaped characters

In [ ]:
track_df["lyrics"].replace("\'", "'", regex=True, inplace=True)
track_df["lyrics"]

# Remove duplicate lyrics track entries

In [ ]:
track_df.drop_duplicates(subset="lyrics", keep=False, inplace=True);
track_df["lyrics"]

# Tag explicit tracks and artists

In [ ]:
artist_df['explicit'] = artist_df['name'].apply(lambda x: has_swear_word(x))

album_df['explicit'] = album_df['name'].apply(lambda x: has_swear_word(x))

track_df['explicitname'] = track_df['name'].apply(lambda x: has_swear_word(x))
track_df['explicittrack'] = track_df['lyrics'].apply(lambda x: has_swear_word(x))
track_df['explicit'] = track_df['explicitname'] | track_df['explicittrack']

track_df.drop(columns=['explicittrack', 'explicitname'], inplace=True)

# Remove artist features from track name

In [ ]:
track_df["name"].replace(r" \(feat\..*\)", "", regex=True, inplace=True)
track_df["name"]

# Save changes

In [ ]:
from sqlalchemy import select, delete, Table, MetaData

con.execute("DELETE from track")
con.execute("DELETE from artist")
con.execute("DELETE from album")

con.execute("ALTER TABLE track ADD COLUMN explicit Boolean")
con.execute("ALTER TABLE artist ADD COLUMN explicit Boolean")
con.execute("ALTER TABLE album ADD COLUMN explicit Boolean")

track_df.to_sql("track", con=con, if_exists='replace', index=False)
artist_df.to_sql("artist", con=con, if_exists='replace', index=False)
album_df.to_sql("album", con=con, if_exists='replace', index=False)

con.execute("PRAGMA foreign_keys = OFF;")

# Remove TrackArtist entries for non-existing tracks
con.execute(
    """
    DELETE FROM track_artist_through
    WHERE track_id NOT IN (
        SELECT id
        FROM track
    );
    """
)

# Remove AlbumArtist entries for non-existing albums
con.execute(
    """
    DELETE FROM album_artist_through
    WHERE album_id NOT IN (
        SELECT id
        FROM album
    );
    """
)

# Remove AlbumTrack entries for non-existing tracks
con.execute(
    """
    DELETE FROM albumtrack
    WHERE track_id NOT IN (
        SELECT id
        FROM track
    );
    """
)

# Remove Album entries for non-existing album tracks
con.execute(
    """
    DELETE FROM album
    WHERE id NOT IN (
        SELECT DISTINCT album_id
        FROM albumtrack
    );
    """
)

# Remove Artist entries for non-existing (track or album) artists
con.execute(
    """
    DELETE FROM artist
    WHERE id NOT IN (
        SELECT DISTINCT artist_id
        FROM track_artist_through 
        UNION
        SELECT DISTINCT artist_id
        FROM album_artist_through
    );
    """
)  

# Close connection
con.close()